In [1]:
from msibi import MSIBI, State, Pair, Bond, Angle
import gsd
import gsd.hoomd
import time
import matplotlib.pyplot as plt
import math
import numpy as np

In [ ]:
%%bash

for dir in "states" "rdfs" "potentials"
do
    if [ -d $dir ]
    then
        rm -r $dir
    fi
done

In [ ]:
cg_traj_files = [
    "4.1kT.gsd",
    "5.5kT.gsd",
    "6.5kT.gsd"
]

In [ ]:
opt = MSIBI(
    integrator="hoomd.md.integrate.nvt",
    integrator_kwargs={"tau": 0.1},
    dt=0.001,
    gsd_period=20000,
    n_steps=2e6,
    max_frames=200,
    verbose=False
)

## Add States

In [ ]:
opt.add_state(
    State(name="A", kT=4.1, traj_file=cg_traj_files[0], alpha=1.0)
)
#opt.add_state(
#    State(name="B", kT=5.5, traj_file=cg_traj_files[1], alpha=0.40)
#)
#opt.add_state(
#    State(name="C", kT=6.2, traj_file=cg_traj_files[2], alpha=0.30)
#)

## Add Pairs

In [ ]:
pair0 = Pair(type1="E", type2="E")
pair1 = Pair(type1="K", type2="K")
pair2 = Pair(type1="E", type2="K")
for pair in [pair0, pair1, pair2]:
    #pair.set_table_potential(epsilon=1, sigma=1, r_min=1e-4, r_max=4.0, n_points=101)
    #pair.set_gauss(epsilon=1, sigma=1.0, r_cut=3.5)
    pair.set_lj(epsilon=1, sigma=1, r_cut=2.5)
    opt.add_pair(pair)

## Add Bonds

In [ ]:
bond0 = Bond(type1="E", type2="K")
bond1 = Bond(type1="K", type2="K")

bond0.set_quadratic(k2=50, l0=1.5, k3=0, k4=0, l_min=0, l_max=3.0, n_points=200)
bond1.set_quadratic(k2=50, l0=1.5, k3=0, k4=0, l_min=0, l_max=3.0, n_points=200)
#bond0.set_harmonic(k=50, l0=1.5)
#bond1.set_harmonic(k=50, l0=1.5)
bond0_init_pot = np.copy(bond0.potential)
bond1_init_pot = np.copy(bond1.potential)
opt.add_bond(bond0)
opt.add_bond(bond1)

## Add Angles

In [ ]:
angle0 = Angle(type1="E", type2="K", type3="K")
angle1 = Angle(type1="K", type2="E", type3="K")

angle0.set_harmonic(k=5, theta0=1.0)
angle1.set_harmonic(k=5, theta0=1.2)

#angle0.set_quadratic(k2=1, theta0=2.3, k3=0, k4=0, n_points=30)
#angle1.set_quadratic(k2=1, theta0=2.3, k3=0, k4=0, n_points=30)

opt.add_angle(angle0)
opt.add_angle(angle1)

## Run Pair Optimization Iterations

In [ ]:
opt.optimize_bonds(n_iterations=1)

In [ ]:
bond = opt.bonds[0]
bond2 = opt.bonds[1]
stateA = opt.states[0]

In [ ]:
plt.plot(bond0.l_range, bond0_init_pot)

In [ ]:
fig = plt.figure()
plt.plot(
    bond._states[stateA]["target_distribution"][:,0],
    bond._states[stateA]["target_distribution"][:,1],
    "o-",
    label="target"
)

plt.plot(
    bond._states[stateA]["current_distribution"][:,0],
    bond._states[stateA]["current_distribution"][:,1],
    "o-",
    label="current"
)

plt.xlim(0.5, 2.5)
plt.legend()

In [ ]:
fig = plt.figure()
plt.plot(bond0.l_range, bond0.potential, label="1")
plt.plot(bond0.l_range, bond0.previous_potential, label="0")
plt.ylim(-1, 100)
plt.legend()

In [ ]:
new_pot = np.copy(bond0_init_pot)

In [ ]:
# If the target dist is zero, do...

In [ ]:
kT = stateA.kT
current_dist = bond._states[stateA]["current_distribution"][:,1]
target_dist = bond._states[stateA]["target_distribution"][:,1]
N = len(bond0._states)

In [ ]:
for i,j in zip(current_dist, target_dist):
    print(i, j)
    print(i/j)
    print()

In [ ]:
real_indices = np.where(np.isfinite(bond.potential))

In [ ]:
np.ediff1d(real_indices)

In [ ]:
np.all(np.ediff1d(real_indices) == 1)

In [ ]:
np.where(np.logical_or(np.isnan(bond.potential), np.isposinf(bond.potential)))

In [ ]:
plt.plot(bond.l_range, bond._states[opt.states[0]]["target_distribution"][:,1])

In [ ]:
for pair in [pair0, pair1, pair2]:
    print(pair._potential_file)
    print(pair.pair_init)
    print(pair.pair_entry)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline
for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,3])
    plt.xlim([0.5,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')
    
plt.title("PM Pair")
#plt.legend()
plt.show()

for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,3])
    plt.xlim([0.5,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-P.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("PP Pair")
#plt.legend()
plt.show()

for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,3])
    plt.xlim([0.5,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.M-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("MM Pair")
#plt.legend()
plt.show()

In [ ]:
%matplotlib inline
for step in range(opt.n_iterations - 1, opt.n_iterations):
    plt.ylim([-1.5,4])
    plt.xlim([0,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')
    
plt.title("PM Pair")
#plt.legend()
plt.show()

for step in range(opt.n_iterations - 1, opt.n_iterations):
    plt.ylim([-1.5,4])
    plt.xlim([0,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.P-P.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("PP Pair")
#plt.legend()
plt.show()

for step in range(opt.n_iterations - 1, opt.n_iterations):
    plt.ylim([-1.5,4])
    plt.xlim([0,5])
    plt.title("MSIBI potentials by iteration")

    step_pot = np.loadtxt(f'potentials/step{step}.pot.M-M.txt')
    plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')

plt.title("MM Pair")
#plt.legend()
plt.show()